# Project 02 - Due Friday, November 11 at 12pm

*Objectives*: Gain experience loading a transactional dataset into a relational database model you define yourself and using SQL to explore its contents. Transform the data into a star schema, documenting the schema visually, and explore the transformed data analytically by writing and executing a number of SQL queries using common syntax and functions and describing your findings.  Gain practice working on these tasks with a partner.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries, diagrams, and charts in the space provided, unless a text answer is requested.  The notebook itself should be completely reproducible at datanotebook.org, from start to finish: another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Attestation*: **Work in pairs**.  At the end of your submitted notebook, identify the work each partner performed and attest that each contributed substantially to the work.

*Deadline*: Friday, November 11, 12pm.  One member of each pair must submit your notebook to Blackboard; you should not both submit it separately.  You may each push a copy to your own GitHub repository.


## Setup - select and obtain data

The US City Open Data Census has a variety of [transactional data from all over the country](http://us-city.census.okfn.org/).  Select one topic and one city from this Census, download the data, and explain your reason for this choice before proceeding.  Provide a link to the web page for the data set you've chosen.

I strongly encourage you to select a dataset from among the following topics:  Crime, Property Assessment, Campaign Finance Contributions, Service Requests (311), and Web Analytics.

By *transactional* data I mean records at the grain of one event per business process.  In the case of Service Requests (311), for example, that would require data at the level of each individual report of a service request.  If this isn't clear, think about the bike trip data - each individual ride was included - and look for data at that level of event/process specificity.  Avoid data like the Connecticut boating data, which was summarized by year, not individual transactions.

Please aim for a dataset of at least 10,000 individual records, but less than 250,000 records.  A little more or a little less is okay, but try to stay within these limits.

### Santa Monica Sevice Requested Dataset
https://data.smgov.net/Public-Services/Closed-GovOutreach-Submissions/tsas-mvez
It is a nice transactional dataset. It has 58268 records, meeting the size requirement. Futhermore, the Santa Monica website provides a nice API for exporting the dataset in a csv format. There at least two dimensions in this dataset, location and time. The "Days to Respond" column is a measurement column in this dataset.



In [4]:
!wget https://data.smgov.net/api/views/tsas-mvez/rows.csv?accessType=DOWNLOAD -O service.csv

--2016-11-02 11:51:04--  https://data.smgov.net/api/views/tsas-mvez/rows.csv?accessType=DOWNLOAD
Resolving data.smgov.net... 52.206.68.26
Connecting to data.smgov.net|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘service.csv’

service.csv             [         <=>        ]   5.71M  1.15MB/s    in 5.3s    

Last-modified header invalid -- time-stamp ignored.
2016-11-02 11:51:09 (1.07 MB/s) - ‘service.csv’ saved [5988947]



In [ ]:
!mkdir prj2data
!mv service.csv ./prj2data

## Problem 1 - examine the data (20 points)

Use `csvstat`, `pandas`, or other tools as you see fit and observe the contents of your dataset.  Review important points from any metadata provided.  Describe what you see:  What do the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some opportunities, challenges, or questions you would like to ask?

In [7]:
!head -n2 ./prj2data/service.csv

Request ID,Topic,Assigned Department,Status,Request Date,Response Date,Days to Respond,Latitude,Longitude,Location
1993315,Public Records Request,City Clerk,Closed,12/15/2014,12/17/2014,3,,,


In [16]:
import pandas as pd
import seaborn as sns
import psycopg2 as pg
%load_ext sql

In [10]:
df = pd.read_csv('./prj2data/service.csv', parse_dates=['Request Date', 'Response Date'])

In [11]:
df.shape

(58268, 10)

#### What do the columns mean?

In [13]:
df.dtypes

Request ID                      int64
Topic                          object
Assigned Department            object
Status                         object
Request Date           datetime64[ns]
Response Date          datetime64[ns]
Days to Respond               float64
Latitude                      float64
Longitude                     float64
Location                       object
dtype: object

The column names are self-explanatory.

#### Are there null values?

In [12]:
df.isnull().sum()

Request ID                 0
Topic                      0
Assigned Department      696
Status                     0
Request Date               0
Response Date              3
Days to Respond            3
Latitude               30723
Longitude              30723
Location               30723
dtype: int64

#### Which columns interest you the most?

Topic, Assigned Department and Days to Respond are the most interesting columns to me. I would like to find out hwo effecient each department, in term of days, handle their service requests.

#### Which columns present some opportunities, challenges, or questions you would like to ask?

Latitude and Longitude have a lot of missing values. Maybe the location data are not relevant for many services. And It is hard to find free service to convert latitudes and longitudes to addresses to enrich the location dimension. 

## Problem 2 - define a database model, load the data, and explore (20 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset as it is in its raw form into the database, and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Explore the data to zero in on a few themes you would like to further study analytically.  Discuss columns that present opportunities for extraction into dimensions, and identify the specific columns that contain facts you want to measure.

In [17]:
!createdb prj2

In [18]:
%sql postgresql://weizheng@localhost:5432/prj2

'Connected: weizheng@prj2'

In [63]:
%%sql
Create Table service(
    request_id integer,
    topic varchar(50),
    assigned_department varchar(50),
    status varchar(50),
    request_date date,
    response_date date,
    days_to_respond real,
    latitude float8,
    longitude float8,
    primary key(request_id)
);

Done.


[]

In [45]:
#drop location because it can be obtained by combining latitude and longtitude
df.drop('Location', axis=1, inplace=True)

In [50]:
len(df.columns)

9

In [48]:
!pwd

/Users/weizheng/msba/data_management/istm-6212


In [52]:
df.to_csv('./prj2data/tosql.csv', index=False)

In [65]:
%%sql
COPY service FROM '/Users/weizheng/msba/data_management/istm-6212/prj2data/tosql.csv'
CSV
HEADER
DELIMITER ',';

58268 rows affected.


[]

In [66]:
%%sql
Select Count(*)
From service;

1 rows affected.


count
58268


## Problem 3 - define a star schema, and load your data into it (30 points)

Define a star schema consisting of at least one fact table and at least two dimensions.  Transform your source data into the new fact table and dimensions.  Discuss your key modeling decisions.

Document your star schema with a diagram.  There several tools available for this, such as Google Docs and Microsoft Visio.

## Problem 4 - explore your data in its new schema (30 points)

Strictly using the dimensional model tables you defined and populated for Problem 3, explore your data in its new form.  Use any of the query strategies we've seen in class, including transformations, aggregates, subqueries, rollups, and, of course, joins.  Add plots to highlight particular themes that stand out.  Describe your thinking and observations along the way.

## Bonus (10 points for one of A or B)


### Option A - Automating ETL

Consider the work you did to load your original raw dataset and then transform it into a dimensional model for analysis.  What would it take to automate this process?  How often would you need to update the data?  Could you easily automate any data cleaning steps?  What checks would you need to put in place to ensure quality?  Would humans need to be involved, or could you automate it all?

Discuss.


### Option B - Augmenting dimensions

Considering your dimensional model, what external data could you find to augment it?  As in the case of bike trips, where weather might provide an interesting added dimension/context not present in the original data, you can probably find another source of data to complement your own model.  Identify one such source and add it to your model, demonstrating its value with a few new queries.